In [5]:
import os
import sys
import pandas as pd
import psycopg2


# Connection parameters
param_dic = {
    "host"      : "localhost",
    "database"  : "globaldata",
    "user"      : "myuser",
    "password"  : "Passw0rd"
}


def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1)
    print("Connection successful")
    return conn


def execute_query(conn, query):
    """ Execute a single query """

    ret = 0 # Return value
    cursor = conn.cursor()
    try:
        cursor.execute(query)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    # If this was a select query, return the result
    if 'select' in query.lower():
        ret = cursor.fetchall()
    cursor.close()
    return ret


def single_inserts(conn, df, table):
    """
    Perform single inserts of the dataframe
    into the PostGIS table
    """
    for i in df.index:
        vals  = [df.at[i,col] for col in list(df.columns)]
        query = """ INSERT INTO %s(name, geom) 
                    VALUES('%s', ST_GeomFromText('POINT(%s %s)', 4326))
                """ % (
            table,
            vals[0],
            vals[1],
            vals[2]
        )
        execute_query(conn, query)



def main():

    # Database connection
    conn = connect(param_dic)
    # Read the csv file
    df = pd.read_csv("./stations.csv")
    print(df.head())
    #Run the execute_many strategy
    single_inserts(conn, df, 'stations')
    # Close the connection
    conn.close()


In [6]:
main()

Connecting to the PostgreSQL database...
Connection successful
                         name  longitude   latitude
0                 Du Quartier -73.433737  45.447199
1                      Panama -73.468134  45.467379
2              Ile-des-Soeurs -73.539361  45.470530
3  Griffintown-Bernard-Landry -73.559108  45.496157
4             Central Station -73.566500  45.501156
